# <center>Деплой модели</center>

Задача развёртывания приложения на сервере называется **деплой** (от англ. deployment — внедрение, развёртывание). По сути, это процесс трансформации исходного кода вашего приложения в рабочее состояние на конкретном сервере.

Для того чтобы внедрить модель в продакшн (в минимальном варианте), необходимо:

1) Сохранить обученную модель в файл.
2) Поднять сервер.
3) Доставить и запустить на нём свою модель.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9cd2438d2cf4036896b4d84c00c5e62a/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_1_3.png)

Опыт подсказывает, что если модель создаётся «в вакууме», то в итоге её просто нельзя будет вывести в продакшн.

Дополнительной проблемой является то, что большинство библиотек для машинного обучения сфокусированы именно на **обучении**, а не на **предсказании**.

# <center>Сохранение и загрузка моделей: pickle и joblib</center>

## <center>Сериализация и десириализация</center>

Чтобы гарантировать сохранение всей структуры данных и получить её при загрузке обратно, используется сериализация.

**Сериализация** — это процесс трансформации любой структуры данных, поддерживаемой в языке, в последовательность битов (или байтов). Обратной операцией является **десериализации**.

In [1]:
# Пример десериализации
line = 'word1, word2, word3'
line.split(",")

['word1', ' word2', ' word3']

In [2]:
# Пример сериализации
",".join(['word1', 'word2', 'word3'])

'word1,word2,word3'

Это простейший пример, но на практике всё гораздо сложнее. Сериализовывать можно не только в текст (как с *CSV*, *JSON* и подобными форматами), но и в бинарный формат, который человек не сможет прочитать.

Бывают форматы, которые могут описывать более сложные структуры (тот же *JSON*). Также можно добавить сжатие итогового набора битов.

> Заметим, что программа должна потратить некоторый ресурс *CPU*, чтобы преобразовать объект в набор байтов и наоборот.

## <center>Инструменты сериализации: Pickle</center>

В стандартную библиотеку *Python* входит модуль `pickle`, который служит для сериализации почти всех объектов произвольного типа.

Мы помним, что объекты находятся в оперативной памяти и направляются в байтовые потоки ввода-вывода. В байтовые потоки может быть направлен любой файлоподобный объект.

**ШАГ №1**

Обучим модель линейной регрессии на встроенном датасете о диабете — *Diabetes dataset*.

В качестве модели, прогнозирующей целевую переменную, возьмём простейшую линейную регрессию, и обучим её на исходных данных:

In [3]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

**ШАГ №2**

Когда мы получили обученную модель, нам необходимо сериализовать её, превратив объект *Python* в поток байтов. Для этого импортируем модуль `pickle` и воспользуемся функцией `dumps()`, в которую нужно передать объект *Python*.

In [4]:
import pickle

# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))

## bytes
## sklearn.linear_model._base.LinearRegression

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


**ШАГ №3**

Попробуем восстановить (десериализовать) объект *Python*. Для этого в модуле `pickle` есть функция `loads()`, в которую нужно передать сериализованный объект (поток байтов).

In [5]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

**ШАГ №4**

Сохраним сериализованный объект прямо в файл. Для этого в `pickle` есть функция `dump()` (без *s* на конце). В неё необходимо передать имя файла или ссылку на открытый файл. Файл назовём *myfile*, его расширение — `.pkl` (формат данных *pickle*):

In [6]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

**ШАГ №5**

Посмотрим на код, который восстанавливает (десериализует) обученную модель из файла *myfile.pkl*. Для этого в *pickle* есть функция `load()` (без *s* на конце). В неё необходимо передать имя файла или ссылку на открытый файл.

In [7]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

**ШАГ №6**

Убедимся, что методы и результаты предсказаний обученной модели и модели, загруженной из файла, совпадают:

In [8]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
all(regressor.predict(X) == regressor_from_bytes.predict(X))
all(regressor.predict(X) == regressor_from_file.predict(X))
## True

True

Как видите, исходная и восстановленная из байтов и файла модели дают одинаковые предсказания. Это значит, что теперь мы можем импортировать наши обученные модели в любое *Python*-приложение и пользоваться ими, минуя этап обучения и все этапы, предшествующие ему.

## <center>Ограничения</center>

Как мы упоминали, у *pickle* есть ограничения. Например, мы не можем сериализовать *лямбда*-функции. Давайте посмотрим, что нам вернёт следующий код:

In [9]:
my_lambda = lambda x: x*2
with open('my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"

PicklingError: Can't pickle <function <lambda> at 0x00000232A92668E0>: attribute lookup <lambda> on __main__ failed

> **Совет**. В таких случаях лучше пользоваться пакетом `dill`.

## <center>Сохранение пайплайна</center>

*Pickle* работает с любыми объектами *Python*. Поэтому для сохранения может быть доступна не просто обученная модель, но и целый **пайплайн**, включающий предобработку данных.

Например, мы хотим сериализовать пайплайн, который включает в себя min-max-нормализацию и отбор пяти наиболее важных факторов на основе корреляции Пирсона. Полученные в результате данные отправляются на вход модели линейной регрессии.

In [10]:
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x00000232EE1CA2A0>)),
                ('Linear', LinearRegression())])

In [11]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [12]:
# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [13]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


> Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции `dumps()` и `loads()`, а не `dump()` и `load()`.

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами *sklearn*. Например, для решения многих задач в нашем курсе мы часто использовали *feature engineering*, чтобы повысить качество работы моделей. Для этого в *sklearn* можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: `TransformerMixin` и `BaseEstimator`.

У трансформера должно быть **три обязательных метода**:

* **`__init__()`** — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров. Например, у трансформера для создания полиномиальных признаков `PolynomialFeatures` из *sklearn* в методе `__init__()` параметр `degree` задаёт степень полинома.

* **`fit()`** — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (*self*).
Например, в трансформере `StandardScaler` в методе `fit()` прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода `fit()`.

* **`transform()`** — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода `transform()` у `StandardScaler` из *sklearn* внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе `fit()`.

Посмотрим на шаблон кастомного трансформера:

In [14]:
from sklearn.base import TransformerMixin, BaseEstimator
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе `transform()` эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы `__init__()` и `fit()` остаются без изменений.

In [15]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

In [17]:
import numpy as np

# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [18]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x00000232EE1CA2A0>)),
                ('Linear', LinearRegression())])

In [19]:
# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [20]:
# Задание 2.5

# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file) 
    
features =np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])
 
# Формируем предсказание
print(np.round(loaded_pipe.predict(features)))

[173.]


## <center>Библиотека Joblib</center>

Иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из *pickle* невозможно восстановить объект полностью.

В таких случаях вместо *pickle* лучше использовать библиотеку `joblib`. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — `.joblib`.

In [21]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')

## ['regr.joblib']

['regr.joblib']

In [22]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True